In [1]:
import pickle
import pandas as pd
import numpy as np
import seaborn as sns
import mne
import matplotlib.pyplot as plt
import pyvista
import ipywidgets
import ipyevents
import pyvistaqt
import yasa
import graphviz
import pydot
import pywt

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, f1_score, roc_curve, auc
from sklearn.utils import class_weight

import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import plot_model

import scipy.signal as signal
from scipy.signal import hilbert
from scipy.signal import stft
from scipy.signal import butter, filtfilt
from scipy.signal import spectrogram

In [2]:
%matplotlib qt

## CNN model

### One-input model

In [3]:
def build_cnn_model_downsampled(input_shape=(300,1)):

    # linear embedding layer
    input_layer = tf.keras.layers.Input(shape=input_shape)

    # Three convolutional blocks (like having three pattern detectors)

    # First convolution block, kernel size of 5
    padded1 = tf.keras.layers.ZeroPadding1D(padding=2)(input_layer)
    conv1 = tf.keras.layers.Conv1D(filters=10, kernel_size=5, strides=1, padding='valid')(padded1)
    # each filter learns a different type of short-time feature
    # stride of 1, moves one step at a time
    conv1 = tf.keras.layers.LeakyReLU(alpha=0.01)(conv1)
    conv1 = tf.keras.layers.MaxPooling1D(pool_size=2)(conv1)
    # K = 2
    conv1 = tf.keras.layers.BatchNormalization()(conv1)

    # Second convolution block, kernel size of 11
    padded2 = tf.keras.layers.ZeroPadding1D(padding=5)(input_layer)
    conv2 = tf.keras.layers.Conv1D(filters=10, kernel_size=11, strides=1, padding='valid')(padded2)
    conv2 = tf.keras.layers.LeakyReLU(alpha=0.01)(conv2)
    conv2 = tf.keras.layers.MaxPooling1D(pool_size=2)(conv2)
    conv2 = tf.keras.layers.BatchNormalization()(conv2)

    # Third convolution block, kernel size of 21
    padded3 = tf.keras.layers.ZeroPadding1D(padding=10)(input_layer)
    conv3 = tf.keras.layers.Conv1D(filters=10, kernel_size=21, strides=1, padding='valid')(padded3)
    conv3 = tf.keras.layers.LeakyReLU(alpha=0.01)(conv3)
    conv3 = tf.keras.layers.MaxPooling1D(pool_size=2)(conv3)
    conv3 = tf.keras.layers.BatchNormalization()(conv3)

    # Concatenate the outputs of all blocks
    concatenated = tf.keras.layers.Concatenate()([conv1, conv2, conv3])

    # GRU Layer
    gru = tf.keras.layers.GRU(64)(concatenated)

    # Fully connected (dense) layer
    dense = tf.keras.layers.Dense(64, activation='relu')(gru)
    # add a Dropout layer to prevent overfitting
    #dense = tf.keras.layers.Dropout(0.5)(dense)

    # Two softmax outputs for dual-task classification
    #output_task1 = tf.keras.layers.Dense(2, activation='softmax', name='task1')(dense)
    #output_task2 = tf.keras.layers.Dense(2, activation='softmax', name='task2')(dense)
    output = tf.keras.layers.Dense(1, activation='sigmoid')(dense)

    # Create the model
    #model = tf.keras.models.Model(inputs=input_layer, outputs=[output_task1, output_task2])
    model = tf.keras.models.Model(inputs=input_layer, outputs=output)

    # Compile the model
    #model.compile(optimizer='adam', loss={'task1': 'categorical_crossentropy', 'task2': 'categorical_crossentropy'}, metrics={'task1': 'accuracy', 'task2': 'accuracy'})
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    # Return the compiled model
    return model

### Two-input model

In [4]:
def build_multi_input_cnn_model_freq():
    # Inputs
    input_raw = tf.keras.Input(shape=(300, 1), name='raw_input')
    input_stft = tf.keras.Input(shape=(13, 1), name='stft_input')  

    def conv_branch(input_layer, kernel_sizes=[5, 11, 21]):
        outputs = []
        for k in kernel_sizes:
            pad = k // 2
            x = tf.keras.layers.ZeroPadding1D(padding=pad)(input_layer)
            x = tf.keras.layers.Conv1D(filters=10, kernel_size=k, strides=1, padding='valid')(x)
            x = tf.keras.layers.LeakyReLU(negative_slope=0.01)(x)
            x = tf.keras.layers.MaxPooling1D(pool_size=2)(x)
            x = tf.keras.layers.BatchNormalization()(x)
            outputs.append(x)
        return tf.keras.layers.Concatenate()(outputs)

    # Convolutional branches
    branch_raw = conv_branch(input_raw)
    branch_stft = conv_branch(input_stft)

    # Each branch through its own GRU
    gru_raw = tf.keras.layers.GRU(64)(branch_raw)
    gru_stft = tf.keras.layers.GRU(64)(branch_stft)

    # Concatenate GRU outputs (fixed-length vectors)
    merged = tf.keras.layers.Concatenate()([gru_raw, gru_stft])

    # Dense layers
    x = tf.keras.layers.Dense(64, activation='relu')(merged)
    output = tf.keras.layers.Dense(1, activation='sigmoid')(x)

    # Build model
    model = tf.keras.Model(inputs=[input_raw, input_stft], outputs=output)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

### Four-input model

In [5]:
def build_four_input_cnn_model_freq():
    # Inputs
    input_raw = tf.keras.Input(shape=(300, 1), name='raw_input')
    input_filtered_so = tf.keras.Input(shape=(300, 1), name='filtered_so')
    input_filtered_spindles = tf.keras.Input(shape=(300, 1), name='filtered_spindles')
    input_stft = tf.keras.Input(shape=(13, 1), name='stft_input')  

    def conv_branch(input_layer, kernel_sizes=[5, 11, 21]):
        outputs = []
        for k in kernel_sizes:
            pad = k // 2
            x = tf.keras.layers.ZeroPadding1D(padding=pad)(input_layer)
            x = tf.keras.layers.Conv1D(filters=10, kernel_size=k, strides=1, padding='valid')(x)
            x = tf.keras.layers.LeakyReLU(alpha=0.01)(x)
            x = tf.keras.layers.MaxPooling1D(pool_size=2)(x)
            x = tf.keras.layers.BatchNormalization()(x)
            outputs.append(x)
        return tf.keras.layers.Concatenate()(outputs)

    # Convolutional branches
    branch_raw = conv_branch(input_raw)
    branch_filtered_so = conv_branch(input_filtered_so)
    branch_filtered_spindles = conv_branch(input_filtered_spindles)
    branch_stft = conv_branch(input_stft)

    # Each branch through its own GRU
    gru_raw = tf.keras.layers.GRU(64)(branch_raw)
    gru_filtered_so = tf.keras.layers.GRU(64)(branch_filtered_so)
    gru_filtered_spindles = tf.keras.layers.GRU(64)(branch_filtered_spindles)
    gru_stft = tf.keras.layers.GRU(64)(branch_stft)

    # Concatenate GRU outputs (fixed-length vectors)
    merged = tf.keras.layers.Concatenate()([gru_raw, gru_filtered_so, gru_filtered_spindles, gru_stft])

    # Dense layers
    x = tf.keras.layers.Dense(64, activation='relu')(merged)
    output = tf.keras.layers.Dense(1, activation='sigmoid')(x)

    # Build model
    model = tf.keras.Model(inputs=[input_raw, input_filtered_so, input_filtered_spindles, input_stft], outputs=output)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

## Detection functions

### Slow oscillations

In [6]:
def detect_slow_oscillations_times(combined_raw, do_filter=True, do_downsample=False, downsample_rate=100):

    # according to methods from Klinzing et al.(2016)

    data = combined_raw.copy().pick_channels(['Fz'])

    if do_filter:
        data.filter(l_freq=0.16, h_freq=1.25)

    if do_downsample:
        data.resample(downsample_rate)
        
    sfreq = data.info['sfreq']
    channel_data = data.get_data()[0]
    
    # 3. find all positive-to-negative zero-crossings
    
    # zero_crossings = np.where( S!= 0)[0]
    # can also save this somewhere for further detection of spindles
    
    S = np.diff(np.sign(channel_data))
    # np.sign returns an array with 1 (positive), 0 (zero), -1 (negative)
    # np.diff calculates the difference between consecutive elements in an array
    # positive value: transition from negative to positive
    # negative value: transition from positive to negative
    # when it's a zero, means that value stayed the same
    zero_crossings = np.where(S < 0)[0]
    # -2 is when a positive-to-negative zero-crossing occurs
    # goes from 1 to -1 
    # -1 - 1 = -2
    # [0] extracts the actual array
    # extracts the indices of interest from current_data (not S)
    #signs = np.sign(current_data)
    #pos_to_neg = np.where((signs[:-1] > 0) & (signs[1:] < 0))[0]
    # detect +1 to -1
    #neg_to_pos = np.where((signs[:-1] <  0) & (signs[1:] > 0))[0]
    # detect -1 to +1

    # 4. Detect peak potentials in each pair
    slow_oscillations = []
    negative_peaks = []
    positive_peaks = []
    peak_to_peak_amplitudes = []
    candidate_indices = []

    # for loop for each pair
    # to collect all the negative and positive peaks
    # to further apply criteria
    count = 0
    for i in range(0, len(zero_crossings)-1, 1):
        # loop through all the zero_crossings
        # step of 1 (with step of 2, miss some zero_crossings)
        start_idx = zero_crossings[i] + 1
        # assigns index of zero-crossing (representing start of potential SO)
        # to start_idx
        end_idx = zero_crossings[i + 1] + 1
        # assigns index of next zero-crossing (representing end of potential SO)
        # to end_idx

        # find the negative to positive crossing in between
        #mid_crossings = neg_to_pos[(neg_to_pos > start_idx) & (neg_to_pos < end_idx)]

        #if len(mid_crossings) != 1:
            #continue

        #mid_idx = mid_crossings [0]

        #duration = (end_idx - start_idx) / sfreq
        #if not (0.8 <= duration <= 2.0):
  
        
        segment_length = (end_idx - start_idx) / sfreq

        # need to add +1 because of way extract segment later

        # have identified index for the pair
        
        # extract data segment between crossings
        
        # find peaks
        if 0.8 <= segment_length <= 2.0:
            count += 1
            segment = channel_data[start_idx:end_idx]
            positive_peak = np.max(segment)
            negative_peak = np.min(segment)
            peak_to_peak_amplitude = positive_peak - negative_peak

        # store values
            candidate_indices.append((start_idx, end_idx))
            positive_peaks.append(positive_peak)
            negative_peaks.append(negative_peak)
            peak_to_peak_amplitudes.append(peak_to_peak_amplitude)

    # calculate mean values for comparison
    #mean_negative_peak = np.mean(negative_peaks)
    # mean_negative_peak = np.mean(negative_peaks) if negative_peaks else 0
    #mean_peak_to_peak_amplitude = np.mean(peak_to_peak_amplitudes)
    # mean_peak_to_peak_amplitude = np.mean(peak_to_peak_amplitudes) if peak_to_peak_amplitudes else 0

    negative_peak_threshold = np.percentile(negative_peaks, 25)
    # keep lowest negative peaks (under the 25th percentile)
    peak_to_peak_amplitude_threshold = np.percentile(peak_to_peak_amplitudes, 75)
    # keep largest peak-to-peak amplitude (over 75th percentile)

    for (start_idx, end_idx), negative_peak, peak_to_peak_amplitude in zip(candidate_indices, negative_peaks, peak_to_peak_amplitudes):
        if peak_to_peak_amplitude >= peak_to_peak_amplitude_threshold and negative_peak <= negative_peak_threshold:
            slow_oscillations.append((start_idx / sfreq, end_idx / sfreq))
            
    return slow_oscillations
    # returns a list of tuples, in which each tuple represents the start and end times of
    # a detected slow oscillation

def detect_slow_oscillations_peaks(combined_raw, do_filter=True, do_downsample=True, downsample_rate=100):

    # according to methods from Klinzing et al.(2016)

    data = combined_raw.copy().pick_channels(['Fz'])

    if do_filter:
        data.filter(l_freq=0.16, h_freq=1.25)

    if do_downsample:
        data.resample(downsample_rate)
        
    sfreq = data.info['sfreq']
    channel_data = data.get_data()[0]
    
    # 3. find all positive-to-negative zero-crossings
    
    # zero_crossings = np.where( S!= 0)[0]
    # can also save this somewhere for further detection of spindles
    
    S = np.diff(np.sign(channel_data))
    # np.sign returns an array with 1 (positive), 0 (zero), -1 (negative)
    # np.diff calculates the difference between consecutive elements in an array
    # positive value: transition from negative to positive
    # negative value: transition from positive to negative
    # when it's a zero, means that value stayed the same
    zero_crossings = np.where(S < 0)[0]
    # -2 is when a positive-to-negative zero-crossing occurs
    # goes from 1 to -1 
    # -1 - 1 = -2
    # [0] extracts the actual array
    # extracts the indices of interest from current_data (not S)


    # 4. Detect peak potentials in each pair
    slow_oscillations = []
    slow_oscillations_peaks = []
    negative_peaks = []
    positive_peaks = []
    peak_to_peak_amplitudes = []
    candidate_indices =  []

    # for loop for each pair
    # to collect all the negative and positive peaks
    # to further apply criteria
    count = 0
    for i in range(0, len(zero_crossings) - 1, 1):
        # loop through all the zero_crossings
        # step of 1 (with step of 2, miss some zero_crossings)
        start_idx = zero_crossings[i] + 1
        # assigns index of zero-crossing (representing start of potential SO)
        # to start_idx
        end_idx = zero_crossings[i + 1] + 1
        # assigns index of next zero-crossing (representing end of potential SO)
        # to end_idx
        segment_length = (end_idx - start_idx) / sfreq

        # need to add +1 because of way extract segment later

        # have identified index for the pair
        
        # extract data segment between crossings
        
        # find peaks
        if 0.8 <= segment_length <= 2.0:
            count += 1
            segment = channel_data[start_idx:end_idx]
            positive_peak = np.max(segment)
            negative_peak = np.min(segment)
            peak_to_peak_amplitude = positive_peak - negative_peak

        # store values
            candidate_indices.append((start_idx, end_idx))
            positive_peaks.append(positive_peak)
            negative_peaks.append(negative_peak)
            peak_to_peak_amplitudes.append(peak_to_peak_amplitude)

    # calculate mean values for comparison
    #mean_negative_peak = np.mean(negative_peaks)
    # mean_negative_peak = np.mean(negative_peaks) if negative_peaks else 0
    #mean_peak_to_peak_amplitude = np.mean(peak_to_peak_amplitudes)
    # mean_peak_to_peak_amplitude = np.mean(peak_to_peak_amplitudes) if peak_to_peak_amplitudes else 0

    negative_peak_threshold = np.percentile(negative_peaks, 25)
    peak_to_peak_amplitude_threshold = np.percentile(peak_to_peak_amplitudes, 75)

    for (start_idx, end_idx), negative_peak, peak_to_peak_amplitude in zip(candidate_indices, negative_peaks, peak_to_peak_amplitudes):
        if peak_to_peak_amplitude >= peak_to_peak_amplitude_threshold and negative_peak <= negative_peak_threshold:
            slow_oscillations.append((start_idx / sfreq, end_idx / sfreq))
            slow_oscillations_peaks.append((negative_peak, positive_peak))

            
    return slow_oscillations
    # returns a list of tuples, in which each tuple represents the start and end times of
    # a detected slow oscillation

In [7]:
def detect_spindles_times(eeg_raw, do_filter=True, do_downsample=False, downsample_rate=100):
    
    # 1. Filter between 12 and 16 Hz
    
    data = eeg_raw.copy().pick_channels(['Fz'])

    if do_filter:
        data.filter(l_freq=12, h_freq=16)
    
    # 2. Downsample at 100 Hz (100 samples per second)

    if do_downsample:
        data.resample(downsample_rate)
    
    sfreq = data.info['sfreq']  
    channel_data = data.get_data()[0]
    # extract the filtered data
    
    
    # 3: Calculate amplitude by applying Hilbert transformation

    hilbert_signal = hilbert(channel_data)
    # apply hilbert transformation to bandpassed data
    # gives analytic signal with amplitude and phase information
    envelope = np.abs(hilbert_signal)
    # take the absolute part of the hilbert signal
    # also the instantaneous power of the signal
    # gives the envelope: amplitude modulation
    # how strength of oscillations change over time
    # size of sliding window
    
    # 4: Perform smoothing with a sliding window of 0.2 seconds
    # this removes high-frequency noise
    
    sliding_window = int(0.2 * sfreq)
    smoothed_envelope = np.convolve(envelope, np.ones(sliding_window) / sliding_window, mode='same')
    # convolving envelope with a uniform filter over the sliding window
    # convolution takes rolling average of 20 samples at a time
    # smooth the signal with the average of values in the window
    # in the smoothed envelope, can detect regions with higher amplitude 
    # which is when a spindle event occurs
    # np.ones: creates a filter kernel
    # have a filter where the sum of all elements equals 1
    # this filter is replaced by the average of the 20 surrounding samples
    # convolution between envelope and averaging filter
    # mode = 'same': so that output of convolution has same length as original envelope

    # 5. Define spindle detection threshold

    threshold = np.percentile(smoothed_envelope, 75)
    spindle_threshold = smoothed_envelope > threshold
    #threshold = np.mean(smoothed_envelope) + 1.5 * np.std(smoothed_envelope)
    #spindle_threshold = smoothed_envelope > threshold
    # threshold is 75th percentile of the smoothed envelope
    # will look at the duration later
    
    # 6. Detect spindles and define peaks and troughs for visualisation
    
    spindles = []
    # initialize list with spindles
    above_threshold = np.where(spindle_threshold)[0]
    # returns indices where signal above the threshold
    stacked_spindles = []
    # initialize list for stacking the spindles for the visualisation
    # contains aligned spindles at peak
    
    if len(above_threshold) > 0:
        # checking it's not empty
        start_idx = above_threshold[0]
        # would be the start of a potential spindle
        for i in range(1, len(above_threshold)):
            if above_threshold[i] > above_threshold[i - 1] + 1:  
                # if above threshold[1] > above_threshold[0] + 1
                # because all indices should be separated by 1
                # so here detects gaps
                # so starting from the second index
                # and comparing each index to the one before
                end_idx = above_threshold[i - 1]
                # so if above condition is true, this is the end of the spindle
                duration = (end_idx - start_idx) / sfreq
                if 0.5 <= duration <= 3:
                    # only keep spindles lasting 0.5 to 3 seconds
                    segment = channel_data[start_idx:end_idx]
                    # extract EEG segment corresponding to detected spindle
                    peak_idx = start_idx + np.argmax(segment) 
                    # extract the peak of the spindle
                    # this will be useful for later
                    spindles.append((start_idx / sfreq, end_idx / sfreq))
                    # all the spindles are stored in spindles
                    
                    # Aligning spindles at peak for visualization
                    before_peak_idx = max(0, peak_idx - int(1.5 * sfreq))
                    # still in the for loop, so this is the peak index of individual peak
                    after_peak_idx = min(len(channel_data), peak_idx + int(1.5 * sfreq))
                    # extracting 1.5 seconds before and after peak
                    # max and min are used for out of bounds situations at the start and end of EEG data
                    aligned_segment = channel_data[before_peak_idx:after_peak_idx]
                    stacked_spindles.append(aligned_segment)
                    # the aligned segment is saved in stacked spindles
                
                start_idx = above_threshold[i]
                # update the start index for the for loop

        # then need to process the final spindle
        end_idx = above_threshold[-1]
        duration = (end_idx - start_idx) / sfreq
        if 0.5 <= duration <= 3:
            segment = channel_data[start_idx:end_idx]
            peak_idx = start_idx + np.argmax(segment)
            spindles.append((start_idx / sfreq, end_idx / sfreq))

            before_peak_idx = max(0, peak_idx - int(1.5 * sfreq))
            after_peak_idx = min(len(channel_data), peak_idx + int(1.5 * sfreq))
            aligned_segment = channel_data[before_peak_idx:after_peak_idx]
            stacked_spindles.append(aligned_segment)
    
    return spindles
    

def detect_spindles_peaks(eeg_raw, do_filter=True, do_downsample=False, downsample_rate=100):
    
    # 1. Filter between 12 and 16 Hz
    
    data = eeg_raw.copy().pick_channels(['Fz'])

    if do_filter:
        data.filter(l_freq=12, h_freq=16)
    
    # 2. Downsample at 100 Hz (100 samples per second)
    
    if do_downsample:
        data.resample(downsample_rate)
        
    sfreq = data.info['sfreq']  
    # update to new sampling frequency
    # because used later in the code
    channel_data = data.get_data()[0]
    # extract the filtered data
    
    # 3: Calculate amplitude by applying Hilbert transformation

    hilbert_signal = hilbert(channel_data)
    # apply hilbert transformation to bandpassed data
    # gives analytic signal with amplitude and phase information
    envelope = np.abs(hilbert_signal)
    # take the absolute part of the hilbert signal
    # also the instantaneous power of the signal
    # gives the envelope: amplitude modulation
    # how strength of oscillations change over time
    # size of sliding window
    
    # 4: Perform smoothing with a sliding window of 0.2 seconds
    # this removes high-frequency noise
    
    sliding_window = int(0.2 * sfreq)
    smoothed_envelope = np.convolve(envelope, np.ones(sliding_window) / sliding_window, mode='same')
    # convolving envelope with a uniform filter over the sliding window
    # convolution takes rolling average of 20 samples at a time
    # smooth the signal with the average of values in the window
    # in the smoothed envelope, can detect regions with higher amplitude 
    # which is when a spindle event occurs
    # np.ones: creates a filter kernel
    # have a filter where the sum of all elements equals 1
    # this filter is replaced by the average of the 20 surrounding samples
    # convolution between envelope and averaging filter
    # mode = 'same': so that output of convolution has same length as original envelope

    # 5. Define spindle detection threshold

    threshold = np.percentile(smoothed_envelope, 75)
    spindle_threshold = smoothed_envelope > threshold
    # 75th percentile as criteria

    #threshold = np.mean(smoothed_envelope) + 1.5 * np.std(smoothed_envelope)
    #spindle_threshold = smoothed_envelope > threshold
    
    # 6. Detect spindles and define peaks and troughs for visualisation
    
    spindles = []
    # initialize list with spindles
    above_threshold = np.where(spindle_threshold)[0]
    # returns indices where signal above the threshold
    stacked_spindles = []
    # initialize list for stacking the spindles for the visualisation
    # contains aligned spindles at peak
    
    if len(above_threshold) > 0:
        # checking it's not empty
        start_idx = above_threshold[0]
        # would be the start of a potential spindle
        for i in range(1, len(above_threshold)):
            if above_threshold[i] > above_threshold[i - 1] + 1:  
                # if above threshold[1] > above_threshold[0] + 1
                # because all indices should be separated by 1
                # so here detects gaps
                end_idx = above_threshold[i - 1]
                # so if above condition is true, this is the end of the spindle
                duration = (end_idx - start_idx) / sfreq
                if 0.5 <= duration <= 3:
                    # only keep spindles lasting 0.5 to 3 seconds
                    segment = channel_data[start_idx:end_idx]
                    # extract EEG segment corresponding to detected spindle
                    peak_idx = start_idx + np.argmax(segment) 
                    # extract the peak of the spindle
                    # this will be useful for later
                    #spindles.append(f"Spindle detected from {start_idx / sfreq:.2f}s to {end_idx / sfreq:.2f}s, peak at {peak_idx / sfreq:.2f}s")
                    spindles.append((peak_idx / sfreq))
                    # all the spindles are stored in spindles
                    
                    # Aligning spindles at peak for visualization
                    before_peak_idx = max(0, peak_idx - int(1.5 * sfreq))
                    # still in the for loop, so this is the peak index of individual peak
                    after_peak_idx = min(len(channel_data), peak_idx + int(1.5 * sfreq))
                    # extracting 1.5 seconds before and after peak
                    # max and min are used for out of bounds situations at the start and end of EEG data
                    aligned_segment = channel_data[before_peak_idx:after_peak_idx]
                    stacked_spindles.append(aligned_segment)
                    # the aligned segment is saved in stacked spindles
                
                start_idx = above_threshold[i]
                # update the start index for the for loop

        # then need to process the final spindle
        end_idx = above_threshold[-1]
        duration = (end_idx - start_idx) / sfreq
        if 0.5 <= duration <= 3:
            segment = channel_data[start_idx:end_idx]
            peak_idx = start_idx + np.argmax(segment)
            spindles.append((peak_idx / sfreq))

            before_peak_idx = max(0, peak_idx - int(1.5 * sfreq))
            after_peak_idx = min(len(channel_data), peak_idx + int(1.5 * sfreq))
            aligned_segment = channel_data[before_peak_idx:after_peak_idx]
            stacked_spindles.append(aligned_segment)

    
    return spindles

In [8]:
def detect_slow_oscillations_spindles_coupling_so_times(combined_raw, do_filter=True, do_downsample=True, downsample_rate=100):
    slow_oscillations_peaks = detect_slow_oscillations_peaks(combined_raw, do_filter=do_filter, do_downsample=do_downsample, downsample_rate=downsample_rate)
    slow_oscillations_times = detect_slow_oscillations_times(combined_raw, do_filter=do_filter, do_downsample=do_downsample, downsample_rate=downsample_rate)
    spindles_peaks = detect_spindles_peaks(combined_raw, do_filter=do_filter, do_downsample=do_downsample, downsample_rate=downsample_rate)

    coupling_times = []
    coupling_times_so = []

    # first detect the coupling events
    for (start_time, end_time), (negative_peak, positive_peak) in zip(slow_oscillations_times, slow_oscillations_peaks):
        for peak in spindles_peaks:
            if negative_peak < peak < end_time:
                coupling_times.append(peak)
                # if the peak of the spindle is between the negative and positive trough
                # add it to list of coupling times

    # then calculate the slow oscillation length
    for start_time, end_time in slow_oscillations_times:
        current_start_time = start_time
        current_end_time = end_time
        for coupling_peak in coupling_times:
            if current_start_time < coupling_peak < current_end_time:
                coupling_times_so.append((current_start_time, current_end_time))

    return coupling_times_so

## Epochs function

In [9]:
def create_fixed_length_epochs(raw, duration=3.0, overlap=0.0, preload=True, reject_by_annotation=False):

    return mne.make_fixed_length_epochs(
        raw,
        duration=duration,
        overlap=overlap,
        preload=preload,
        reject_by_annotation=reject_by_annotation
    )
# function mne.make_fixed_length_epochs takes into account the sampling frequency of the data

def label_coupling_epochs(epochs, coupling_starts, coupling_ends, epoch_length_sec=3.0):

    epoch_starts = np.arange(len(epochs)) * epoch_length_sec
    # new np array with the start time of each epoch
    # epoch_starts[i] is the start time of each epoch

    epoch_labels = np.zeros(len(epochs), dtype=int)
    # initialize all the labels as 0 initially

    for start, end in zip(coupling_starts, coupling_ends):
        # loop through the start and end times of detected spindles by YASA
        for i, epoch_start in enumerate(epoch_starts):
            # loop through the one-second epochs that are not labelled yet
            epoch_end = epoch_start + epoch_length_sec
            # for each epoch, calculate the epoch end time
            # which is epoch_start + length of epoch
            # so now have the time range of each epoch
            if (start < epoch_end) and (end > epoch_start):
                # if the spindle started before the epoch ends
                # and the spindle ended after the epoch started
                epoch_labels[i] = 1
                
    return epoch_labels

def label_coupling_epochs_moderate(epochs, coupling_starts, coupling_ends, epoch_length_sec=3.0):
    epoch_starts = np.arange(len(epochs)) * epoch_length_sec
    epoch_labels = np.zeros(len(epochs), dtype=int)

    for coupling_start, coupling_end in zip(coupling_starts, coupling_ends):
        coupling_duration = coupling_end - coupling_start
        required_overlap = 0.5 * coupling_duration  
        # only label 1 if epoch contains 50% of the SO duration

        for i, epoch_start in enumerate(epoch_starts):
            epoch_end = epoch_start + epoch_length_sec

            # Calculate overlap between coupling and epoch
            overlap_start = max(coupling_start, epoch_start)
            overlap_end = min(coupling_end, epoch_end)
            overlap_duration = overlap_end - overlap_start

            if overlap_duration >= required_overlap:
                epoch_labels[i] = 1

    return epoch_labels

## Importing data

In [10]:
# file paths
train_file = r"C:\EEG DATA\combined_sets\train_raw.fif"
test_file = r"C:\EEG DATA\combined_sets\test_raw.fif"

# load raw files
train_raw = mne.io.read_raw_fif(train_file, preload=True)
test_raw = mne.io.read_raw_fif(test_file, preload=True)

Opening raw data file C:\EEG DATA\combined_sets\train_raw.fif...
Isotrak not found
    Range : 1470000 ... 23295072 =   2940.000 ... 46590.144 secs
Ready.
Reading 0 ... 21825072  =      0.000 ... 43650.144 secs...
Opening raw data file C:\EEG DATA\combined_sets\test_raw.fif...
Isotrak not found
    Range : 825000 ... 17985049 =   1650.000 ... 35970.098 secs
Ready.
Reading 0 ... 17160049  =      0.000 ... 34320.098 secs...


## With raw data

### Coupling detection

In [11]:
coupling_train_times_raw_downsampled = detect_slow_oscillations_spindles_coupling_so_times(train_raw, do_filter=True, do_downsample=True)
coupling_test_times_raw_downsampled = detect_slow_oscillations_spindles_coupling_so_times(test_raw, do_filter=True, do_downsample=True)

coupling_starts_train_raw_downsampled, coupling_ends_train_raw_downsampled = zip(*coupling_train_times_raw_downsampled) if coupling_train_times_raw_downsampled else([],[])
coupling_starts_test_raw_downsampled, coupling_ends_test_raw_downsampled = zip(*coupling_test_times_raw_downsampled) if coupling_test_times_raw_downsampled else([],[])

print(len(coupling_starts_train_raw_downsampled))
print(len(coupling_ends_train_raw_downsampled))

print(len(coupling_starts_test_raw_downsampled))
print(len(coupling_ends_test_raw_downsampled))

NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
Filtering raw data in 73 contiguous segments
Setting up band-pass filter from 0.16 - 1.2 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.16
- Lower transition bandwidth: 0.16 Hz (-6 dB cutoff frequency: 0.08 Hz)
- Upper passband edge: 1.25 Hz
- Upper transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 2.25 Hz)
- Filter length: 10313 samples (20.626 s)

NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
Filtering raw data in 73 contiguous segments
Setting up band-pass filter from 0.16 - 1.2 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194

### Downsample

In [12]:
train_raw_downsampled = train_raw.copy().resample(100)
test_raw_downsampled = test_raw.copy().resample(100)

print(train_raw_downsampled.info['sfreq'])
print(test_raw_downsampled.info['sfreq'])

100.0
100.0


### Epoch the data

In [13]:
epochs_train_raw_downsampled = create_fixed_length_epochs(train_raw_downsampled)
epochs_test_raw_downsampled = create_fixed_length_epochs(test_raw_downsampled)
print(epochs_train_raw_downsampled.get_data().shape)
print(epochs_test_raw_downsampled.get_data().shape)

Not setting metadata
14550 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 14550 events and 300 original time points ...
0 bad epochs dropped
Not setting metadata
11440 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 11440 events and 300 original time points ...
0 bad epochs dropped
(14550, 1, 300)
(11440, 1, 300)


#### no threshold

In [14]:
%%time

# Train set

epoch_labels_train_raw_downsampled = label_coupling_epochs(epochs_train_raw_downsampled, coupling_starts_train_raw_downsampled, coupling_ends_train_raw_downsampled)

print(f"Train data first 60 coupling events: {coupling_starts_train_raw_downsampled[:60]}")
print(f"Train labels first 60 epochs: {epoch_labels_train_raw_downsampled[:60]}")

# Test set

epoch_labels_test_raw_downsampled = label_coupling_epochs(epochs_test_raw_downsampled, coupling_starts_test_raw_downsampled, coupling_ends_test_raw_downsampled)

print(f"\nTest data first 60 coupling events: {coupling_starts_test_raw_downsampled[:60]}")
print(f"Test labels first 60 epochs: {epoch_labels_test_raw_downsampled[:60]}")

Train data first 60 coupling events: (np.float64(83.41), np.float64(108.08), np.float64(262.01), np.float64(274.78), np.float64(378.52), np.float64(482.73), np.float64(502.23), np.float64(567.15), np.float64(599.39), np.float64(600.94), np.float64(685.84), np.float64(697.85), np.float64(707.43), np.float64(708.93), np.float64(720.49), np.float64(732.11), np.float64(776.09), np.float64(780.06), np.float64(864.6), np.float64(871.32), np.float64(910.43), np.float64(917.07), np.float64(960.41), np.float64(1050.52), np.float64(1200.05), np.float64(1262.04), np.float64(1278.35), np.float64(1330.87), np.float64(1356.47), np.float64(1371.44), np.float64(1395.83), np.float64(1449.16), np.float64(1566.73), np.float64(1615.49), np.float64(1616.67), np.float64(1973.78), np.float64(2039.95), np.float64(2091.33), np.float64(2121.41), np.float64(2244.99), np.float64(2244.99), np.float64(2367.83), np.float64(2368.8), np.float64(2440.8), np.float64(2445.04), np.float64(2565.68), np.float64(2910.09), np

#### 50% threshold (moderate)

In [15]:
%%time

# Train set

epoch_labels_train_raw_downsampled_moderate = label_coupling_epochs_moderate(epochs_train_raw_downsampled, coupling_starts_train_raw_downsampled, coupling_ends_train_raw_downsampled)

print(f"Train data first 60 coupling events: {coupling_starts_train_raw_downsampled[:60]}")
print(f"Train labels first 60 epochs: {epoch_labels_train_raw_downsampled_moderate[:60]}")

# Test set

epoch_labels_test_raw_downsampled_moderate = label_coupling_epochs_moderate(epochs_test_raw_downsampled, coupling_starts_test_raw_downsampled, coupling_ends_test_raw_downsampled)

print(f"\nTest data first 60 coupling events: {coupling_starts_test_raw_downsampled[:60]}")
print(f"Test labels first 60 epochs: {epoch_labels_test_raw_downsampled_moderate[:60]}")

Train data first 60 coupling events: (np.float64(83.41), np.float64(108.08), np.float64(262.01), np.float64(274.78), np.float64(378.52), np.float64(482.73), np.float64(502.23), np.float64(567.15), np.float64(599.39), np.float64(600.94), np.float64(685.84), np.float64(697.85), np.float64(707.43), np.float64(708.93), np.float64(720.49), np.float64(732.11), np.float64(776.09), np.float64(780.06), np.float64(864.6), np.float64(871.32), np.float64(910.43), np.float64(917.07), np.float64(960.41), np.float64(1050.52), np.float64(1200.05), np.float64(1262.04), np.float64(1278.35), np.float64(1330.87), np.float64(1356.47), np.float64(1371.44), np.float64(1395.83), np.float64(1449.16), np.float64(1566.73), np.float64(1615.49), np.float64(1616.67), np.float64(1973.78), np.float64(2039.95), np.float64(2091.33), np.float64(2121.41), np.float64(2244.99), np.float64(2244.99), np.float64(2367.83), np.float64(2368.8), np.float64(2440.8), np.float64(2445.04), np.float64(2565.68), np.float64(2910.09), np

In [16]:
print("With no threshold:")
# Train set
train_labels = epoch_labels_train_raw_downsampled
train_counts = np.bincount(train_labels)
train_total = len(train_labels)

print("\nTrain label distribution:")
for label, count in enumerate(train_counts):
    proportion = count / train_total
    print(f"Label {label}: count = {count}, proportion = {proportion:.2%}")

# Test set
test_labels = epoch_labels_test_raw_downsampled
test_counts = np.bincount(test_labels)
test_total = len(test_labels)

print("\nTest label distribution:")
for label, count in enumerate(test_counts):
    proportion = count / test_total
    print(f"Label {label}: count = {count}, proportion = {proportion:.2%}")

With no threshold:

Train label distribution:
Label 0: count = 13029, proportion = 89.55%
Label 1: count = 1521, proportion = 10.45%

Test label distribution:
Label 0: count = 10273, proportion = 89.80%
Label 1: count = 1167, proportion = 10.20%


In [17]:
print("With 50% threshold:")
# Train set
train_labels = epoch_labels_train_raw_downsampled_moderate
train_counts = np.bincount(train_labels)
train_total = len(train_labels)

print("\nTrain label distribution:")
for label, count in enumerate(train_counts):
    proportion = count / train_total
    print(f"Label {label}: count = {count}, proportion = {proportion:.2%}")

# Test set
test_labels = epoch_labels_test_raw_downsampled_moderate
test_counts = np.bincount(test_labels)
test_total = len(test_labels)

print("\nTest label distribution:")
for label, count in enumerate(test_counts):
    proportion = count / test_total
    print(f"Label {label}: count = {count}, proportion = {proportion:.2%}")

With 50% threshold:

Train label distribution:
Label 0: count = 13463, proportion = 92.53%
Label 1: count = 1087, proportion = 7.47%

Test label distribution:
Label 0: count = 10605, proportion = 92.70%
Label 1: count = 835, proportion = 7.30%


### Prepare EEG data for CNN inputs

#### X and y train and test sets

In [18]:
# Reshape arrays

epochs_train_np_raw_downsampled = np.array(epochs_train_raw_downsampled).reshape(len(epochs_train_raw_downsampled), -1, 1)
# number of epochs N, sampling frequency (time dimension automatically inferred), channel dimension
epochs_test_np_raw_downsampled = np.array(epochs_test_raw_downsampled).reshape(len(epochs_test_raw_downsampled), -1, 1)

### Define X and Y sets

In [19]:
# Define X and y sets
                                                                 
X_train_raw_downsampled = epochs_train_np_raw_downsampled
y_train_raw_downsampled = epoch_labels_train_raw_downsampled_moderate

X_test_raw_downsampled = epochs_test_np_raw_downsampled
y_test_raw_downsampled = epoch_labels_test_raw_downsampled_moderate

# Print shapes

print(f"X_train shape: {X_train_raw_downsampled.shape}")
print(f"y_train shape: {y_train_raw_downsampled.shape}")

print(f"\nX_test shape: {X_test_raw_downsampled.shape}")
print(f"y_test shape: {y_test_raw_downsampled.shape}")

X_train shape: (14550, 300, 1)
y_train shape: (14550,)

X_test shape: (11440, 300, 1)
y_test shape: (11440,)


### Train the model

In [20]:
early_stop = EarlyStopping(
    monitor='val_loss',      
    patience=5,               
    restore_best_weights=True 
)
# stop after 5 epochs with no improvement

In [21]:
# show model architecture
input_shape = (300, 1)
model_raw_downsampled_coupling = build_cnn_model_downsampled(input_shape)
model_raw_downsampled_coupling.summary()

C:\Users\roseb\anaconda3\envs\msc_research_project\Lib\site-packages\keras\src\layers\activations\leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)      │ (None, 300, 1)            │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ zero_padding1d                │ (None, 304, 1)            │               0 │ input_layer[0][0]          │
│ (ZeroPadding1D)               │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ zero_padding1d_1              │ (None, 310, 1)            │               0 │ input_layer[0][0]          │
│ (ZeroPadding1D)               │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ zero_padding1d_2              │ (None, 320, 1)            │               0 │ input_layer[0][0]          │
│ (ZeroPadding1D)               │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv1d (Conv1D)               │ (None, 300, 10)           │              60 │ zero_padding1d[0][0]       │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv1d_1 (Conv1D)             │ (None, 300, 10)           │             120 │ zero_padding1d_1[0][0]     │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv1d_2 (Conv1D)             │ (None, 300, 10)           │             220 │ zero_padding1d_2[0][0]     │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ leaky_re_lu (LeakyReLU)       │ (None, 300, 10)           │               0 │ conv1d[0][0]               │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ leaky_re_lu_1 (LeakyReLU)     │ (None, 300, 10)           │               0 │ conv1d_1[0][0]             │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ leaky_re_lu_2 (LeakyReLU)     │ (None, 300, 10)           │               0 │ conv1d_2[0][0]             │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ max_pooling1d (MaxPooling1D)  │ (None, 150, 10)           │               0 │ leaky_re_lu[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ max_pooling1d_1               │ (None, 150, 10)           │               0 │ leaky_re_lu_1[0][0]        │
│ (MaxPooling1D)                │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ max_pooling1d_2               │ (None, 150, 10)           │               0 │ leaky_re_lu_2[0][0]        │
│ (MaxPooling1D)                │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization           │ (None, 150, 10)           │              40 │ max_pooling1d[0][0]        │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼───────────────

 Total params: 23,177 (90.54 KB)

 Trainable params: 23,117 (90.30 KB)

 Non-trainable params: 60 (240.00 B)

In [22]:
%%time

training_info_raw_downsampled_coupling = model_raw_downsampled_coupling.fit(X_train_raw_downsampled, y_train_raw_downsampled, validation_split=0.2, epochs=20, batch_size=128, callbacks=[early_stop])

Epoch 1/20
91/91 ━━━━━━━━━━━━━━━━━━━━ 26s 164ms/step - accuracy: 0.8845 - loss: 0.3725 - val_accuracy: 0.9419 - val_loss: 0.2321
Epoch 2/20
91/91 ━━━━━━━━━━━━━━━━━━━━ 13s 138ms/step - accuracy: 0.9171 - loss: 0.2742 - val_accuracy: 0.9419 - val_loss: 0.2033
Epoch 3/20
91/91 ━━━━━━━━━━━━━━━━━━━━ 11s 119ms/step - accuracy: 0.9223 - loss: 0.2448 - val_accuracy: 0.9419 - val_loss: 0.1649
Epoch 4/20
91/91 ━━━━━━━━━━━━━━━━━━━━ 11s 119ms/step - accuracy: 0.9248 - loss: 0.2073 - val_accuracy: 0.9419 - val_loss: 0.1427
Epoch 5/20
91/91 ━━━━━━━━━━━━━━━━━━━━ 11s 116ms/step - accuracy: 0.9213 - loss: 0.1928 - val_accuracy: 0.9419 - val_loss: 0.1448
Epoch 6/20
91/91 ━━━━━━━━━━━━━━━━━━━━ 10s 111ms/step - accuracy: 0.9218 - loss: 0.1916 - val_accuracy: 0.9419 - val_loss: 0.1413
Epoch 7/20
91/91 ━━━━━━━━━━━━━━━━━━━━ 11s 118ms/step - accuracy: 0.9201 - loss: 0.1881 - val_accuracy: 0.9419 - val_loss: 0.1430
Epoch 8/20
91/91 ━━━━━━━━━━━━━━━━━━━━ 12s 127ms/step - accuracy: 0.9223 - loss: 0.1820 - val_accu

In [23]:
def plot_training_history(training_info):
    fig, axs = plt.subplots(1, 2, figsize=(16, 5))
    axs[0].plot(training_info.history['loss'], label="training set")
    axs[0].plot(training_info.history['val_loss'], label="validation set")
    axs[0].set_xlabel("Epoch")
    axs[0].set_ylabel("Loss")
    axs[0].grid(True)
    axs[0].legend()
    try:
        axs[1].plot(training_info.history['accuracy'], label="training set")
        axs[1].plot(training_info.history['val_accuracy'], label="validation set")
        axs[1].set_xlabel("Epoch")
        axs[1].set_ylabel("Accuracy")
        axs[1].grid(True)
        axs[1].legend()
    except:
        pass
  
    fig.suptitle("Training history for one-input CNN model with raw EEG data for coupling detection", fontsize=16)
    plt.tight_layout(rect=[0, 0, 1, 0.95])
    plt.show()

plot_training_history(training_info_raw_downsampled_coupling)

In [24]:
model_raw_downsampled_coupling.evaluate(X_test_raw_downsampled, y_test_raw_downsampled)

358/358 ━━━━━━━━━━━━━━━━━━━━ 6s 16ms/step - accuracy: 0.9221 - loss: 0.1910


[0.18005529046058655, 0.931381106376648]

In [25]:
y_pred = model_raw_downsampled_coupling.predict(X_test_raw_downsampled)
y_pred_labels = (y_pred > 0.5).astype(int)

print(confusion_matrix(y_test_raw_downsampled, y_pred_labels))
print(classification_report(y_test_raw_downsampled, y_pred_labels))

358/358 ━━━━━━━━━━━━━━━━━━━━ 6s 16ms/step
[[10491   114]
 [  671   164]]
              precision    recall  f1-score   support

           0       0.94      0.99      0.96     10605
           1       0.59      0.20      0.29       835

    accuracy                           0.93     11440
   macro avg       0.76      0.59      0.63     11440
weighted avg       0.91      0.93      0.92     11440



In [26]:
# Get predictions
y_pred = model_raw_downsampled_coupling.predict(X_test_raw_downsampled)
y_pred_labels = (y_pred > 0.5).astype(int)

# Confusion matrix
cm = confusion_matrix(y_test_raw_downsampled, y_pred_labels)
cm_df = pd.DataFrame(cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

# Classification report as a dataframe
report = classification_report(y_test_raw_downsampled, y_pred_labels, output_dict=True)
report_df = pd.DataFrame(report).transpose()

# Confusion matrix plotted as heatmap
plt.figure(figsize=(6, 5))
sns.heatmap(cm_df, annot=True, fmt="d", cmap="Blues")
plt.title("Confusion Matrix for Coupling Detection \nusing a one-input CNN with raw EEG data")
plt.ylabel("True Label")
plt.xlabel("Predicted Label")
plt.tight_layout()
plt.show()

# Classification report as a table
fig, ax = plt.subplots(figsize=(10, 4))
ax.axis('tight')
ax.axis('off')
table = ax.table(cellText=report_df.round(2).values,
                 colLabels=report_df.columns,
                 rowLabels=report_df.index,
                 cellLoc='center',
                 loc='center')
table.auto_set_font_size(False)
table.set_fontsize(10)
table.scale(1.2, 1.2)
plt.title("Classification Report for Coupling Detection \nusing a one-input CNN with raw EEG data", fontsize=14)
plt.tight_layout()
plt.show()


358/358 ━━━━━━━━━━━━━━━━━━━━ 6s 17ms/step


In [27]:
# Flatten in case y_pred has shape (n_samples, 1)
y_pred_proba = y_pred.ravel()

# Compute ROC curve and AUC
fpr, tpr, thresholds = roc_curve(y_test_raw_downsampled, y_pred_proba)
roc_auc = auc(fpr, tpr)

# Plotting
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f"ROC curve (AUC = {roc_auc:.2f})")
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Random chance')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Receiver Operating Characteristic for SO Detection \nusing a one-input CNN with raw EEG data")
plt.legend(loc="lower right")
plt.grid(True)
plt.tight_layout()
plt.show()

## With SO filtered data (0.16-1.25 Hz)

### Coupling detection

In [28]:
# this is not necessary, could be reusing

coupling_train_times_filtered_downsampled_so = detect_slow_oscillations_spindles_coupling_so_times(train_raw, do_filter=True, do_downsample=True)
coupling_test_times_filtered_downsampled_so = detect_slow_oscillations_spindles_coupling_so_times(test_raw, do_filter=True, do_downsample=True)
# since filtering and downsampling before, do not filter and downsample again in function

coupling_starts_train_filtered_downsampled_so, coupling_ends_train_filtered_downsampled_so = zip(*coupling_train_times_filtered_downsampled_so) if coupling_train_times_filtered_downsampled_so else([],[])
coupling_starts_test_filtered_downsampled_so, coupling_ends_test_filtered_downsampled_so = zip(*coupling_test_times_filtered_downsampled_so) if coupling_test_times_filtered_downsampled_so else([],[])

print(len(coupling_starts_train_filtered_downsampled_so))
print(len(coupling_ends_train_filtered_downsampled_so))

print(len(coupling_starts_test_filtered_downsampled_so))
print(len(coupling_ends_test_filtered_downsampled_so))

NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
Filtering raw data in 73 contiguous segments
Setting up band-pass filter from 0.16 - 1.2 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.16
- Lower transition bandwidth: 0.16 Hz (-6 dB cutoff frequency: 0.08 Hz)
- Upper passband edge: 1.25 Hz
- Upper transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 2.25 Hz)
- Filter length: 10313 samples (20.626 s)

NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
Filtering raw data in 73 contiguous segments
Setting up band-pass filter from 0.16 - 1.2 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194

In [29]:
# Apply bandpass filter between 0.16 and 1.25 Hz
# to compare performance of filtered dataset to unfiltered one
train_filtered_downsampled_so = train_raw.copy().filter(l_freq=0.16, h_freq=1.25)
test_filtered_downsampled_so = test_raw.copy().filter(l_freq=0.16, h_freq=1.25)

# Downsample to 100 Hz
train_filtered_downsampled_so.resample(100)
test_filtered_downsampled_so.resample(100)

Filtering raw data in 73 contiguous segments
Setting up band-pass filter from 0.16 - 1.2 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.16
- Lower transition bandwidth: 0.16 Hz (-6 dB cutoff frequency: 0.08 Hz)
- Upper passband edge: 1.25 Hz
- Upper transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 2.25 Hz)
- Filter length: 10313 samples (20.626 s)

Filtering raw data in 50 contiguous segments
Setting up band-pass filter from 0.16 - 1.2 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.16
- Lower transition bandwidth: 0.16 Hz (-6 dB cutoff frequency: 0.08 Hz)
- Upper pass

<Raw | test_raw.fif, 1 x 3432010 (34320.1 s), ~26.2 MiB, data loaded>

### Epoch the data

In [30]:
epochs_train_filtered_downsampled_so = create_fixed_length_epochs(train_filtered_downsampled_so)
epochs_test_filtered_downsampled_so = create_fixed_length_epochs(test_filtered_downsampled_so)
print(epochs_train_filtered_downsampled_so.get_data().shape)
print(epochs_test_filtered_downsampled_so.get_data().shape)

Not setting metadata
14550 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 14550 events and 300 original time points ...
0 bad epochs dropped
Not setting metadata
11440 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 11440 events and 300 original time points ...
0 bad epochs dropped
(14550, 1, 300)
(11440, 1, 300)


### Labels for 3-second epochs

In [31]:
%%time

# Train set

epoch_labels_train_filtered_downsampled_so = label_coupling_epochs(epochs_train_filtered_downsampled_so, coupling_starts_train_filtered_downsampled_so, coupling_ends_train_filtered_downsampled_so)

print(f"Train data first 60 coupling events: {coupling_starts_train_filtered_downsampled_so[:60]}")
print(f"Train labels first 60 epochs: {epoch_labels_train_filtered_downsampled_so[:60]}")

# Test set

epoch_labels_test_filtered_downsampled_so = label_coupling_epochs(epochs_test_filtered_downsampled_so, coupling_starts_test_filtered_downsampled_so, coupling_ends_test_filtered_downsampled_so)

print(f"\nTest data first 60 coupling events: {coupling_starts_test_filtered_downsampled_so[:60]}")
print(f"Test labels first 60 epochs: {epoch_labels_test_filtered_downsampled_so[:60]}")

Train data first 60 coupling events: (np.float64(83.41), np.float64(108.08), np.float64(262.01), np.float64(274.78), np.float64(378.52), np.float64(482.73), np.float64(502.23), np.float64(567.15), np.float64(599.39), np.float64(600.94), np.float64(685.84), np.float64(697.85), np.float64(707.43), np.float64(708.93), np.float64(720.49), np.float64(732.11), np.float64(776.09), np.float64(780.06), np.float64(864.6), np.float64(871.32), np.float64(910.43), np.float64(917.07), np.float64(960.41), np.float64(1050.52), np.float64(1200.05), np.float64(1262.04), np.float64(1278.35), np.float64(1330.87), np.float64(1356.47), np.float64(1371.44), np.float64(1395.83), np.float64(1449.16), np.float64(1566.73), np.float64(1615.49), np.float64(1616.67), np.float64(1973.78), np.float64(2039.95), np.float64(2091.33), np.float64(2121.41), np.float64(2244.99), np.float64(2244.99), np.float64(2367.83), np.float64(2368.8), np.float64(2440.8), np.float64(2445.04), np.float64(2565.68), np.float64(2910.09), np

In [32]:
%%time

# Train set

epoch_labels_train_filtered_downsampled_so_moderate = label_coupling_epochs_moderate(epochs_train_filtered_downsampled_so, coupling_starts_train_filtered_downsampled_so, coupling_ends_train_filtered_downsampled_so)

print(f"Train data first 60 coupling events: {coupling_starts_train_filtered_downsampled_so[:60]}")
print(f"Train labels first 60 epochs: {epoch_labels_train_filtered_downsampled_so_moderate[:60]}")

# Test set

epoch_labels_test_filtered_downsampled_so_moderate = label_coupling_epochs_moderate(epochs_test_filtered_downsampled_so, coupling_starts_test_filtered_downsampled_so, coupling_ends_test_filtered_downsampled_so)

print(f"\nTest data first 60 coupling events: {coupling_starts_test_filtered_downsampled_so[:60]}")
print(f"Test labels first 60 epochs: {epoch_labels_test_filtered_downsampled_so_moderate[:60]}")

Train data first 60 coupling events: (np.float64(83.41), np.float64(108.08), np.float64(262.01), np.float64(274.78), np.float64(378.52), np.float64(482.73), np.float64(502.23), np.float64(567.15), np.float64(599.39), np.float64(600.94), np.float64(685.84), np.float64(697.85), np.float64(707.43), np.float64(708.93), np.float64(720.49), np.float64(732.11), np.float64(776.09), np.float64(780.06), np.float64(864.6), np.float64(871.32), np.float64(910.43), np.float64(917.07), np.float64(960.41), np.float64(1050.52), np.float64(1200.05), np.float64(1262.04), np.float64(1278.35), np.float64(1330.87), np.float64(1356.47), np.float64(1371.44), np.float64(1395.83), np.float64(1449.16), np.float64(1566.73), np.float64(1615.49), np.float64(1616.67), np.float64(1973.78), np.float64(2039.95), np.float64(2091.33), np.float64(2121.41), np.float64(2244.99), np.float64(2244.99), np.float64(2367.83), np.float64(2368.8), np.float64(2440.8), np.float64(2445.04), np.float64(2565.68), np.float64(2910.09), np

In [33]:
print("With 50% threshold:")
# Train set
train_labels = epoch_labels_train_filtered_downsampled_so_moderate
train_counts = np.bincount(train_labels)
train_total = len(train_labels)

print("\nTrain label distribution:")
for label, count in enumerate(train_counts):
    proportion = count / train_total
    print(f"Label {label}: count = {count}, proportion = {proportion:.2%}")

# Test set
test_labels = epoch_labels_test_filtered_downsampled_so_moderate
test_counts = np.bincount(test_labels)
test_total = len(test_labels)

print("\nTest label distribution:")
for label, count in enumerate(test_counts):
    proportion = count / test_total
    print(f"Label {label}: count = {count}, proportion = {proportion:.2%}")

With 50% threshold:

Train label distribution:
Label 0: count = 13463, proportion = 92.53%
Label 1: count = 1087, proportion = 7.47%

Test label distribution:
Label 0: count = 10605, proportion = 92.70%
Label 1: count = 835, proportion = 7.30%


### Prepare EEG data for CNN input

#### X and y train and test sets

In [34]:
# Reshape arrays

epochs_train_np_filtered_downsampled_so = np.array(epochs_train_filtered_downsampled_so).reshape(len(epochs_train_filtered_downsampled_so), -1, 1)
# number of epochs N, sampling frequency (time dimension automatically inferred), channel dimension
epochs_test_np_filtered_downsampled_so = np.array(epochs_test_filtered_downsampled_so).reshape(len(epochs_test_filtered_downsampled_so), -1, 1)

In [35]:
# Define X and y sets
                                                                 
X_train_filtered_downsampled_so = epochs_train_np_filtered_downsampled_so
y_train_filtered_downsampled_so = epoch_labels_train_filtered_downsampled_so_moderate

X_test_filtered_downsampled_so = epochs_test_np_filtered_downsampled_so
y_test_filtered_downsampled_so = epoch_labels_test_filtered_downsampled_so_moderate

# Print shapes

print(f"X_train shape: {X_train_filtered_downsampled_so.shape}")
print(f"y_train shape: {y_train_filtered_downsampled_so.shape}")

print(f"\nX_test shape: {X_test_filtered_downsampled_so.shape}")
print(f"y_test shape: {y_test_filtered_downsampled_so.shape}")

X_train shape: (14550, 300, 1)
y_train shape: (14550,)

X_test shape: (11440, 300, 1)
y_test shape: (11440,)


## With spindle filtered data (12-16 Hz)

In [36]:
# this is not necessary, could be reusing

coupling_train_times_filtered_downsampled_spindles = detect_slow_oscillations_spindles_coupling_so_times(train_raw, do_filter=True, do_downsample=True)
coupling_test_times_filtered_downsampled_spindles = detect_slow_oscillations_spindles_coupling_so_times(test_raw, do_filter=True, do_downsample=True)
# since filtering and downsampling before, do not filter and downsample again in function

coupling_starts_train_filtered_downsampled_spindles, coupling_ends_train_filtered_downsampled_spindles = zip(*coupling_train_times_filtered_downsampled_spindles) if coupling_train_times_filtered_downsampled_spindles else([],[])
coupling_starts_test_filtered_downsampled_spindles, coupling_ends_test_filtered_downsampled_spindles = zip(*coupling_test_times_filtered_downsampled_spindles) if coupling_test_times_filtered_downsampled_spindles else([],[])

print(len(coupling_starts_train_filtered_downsampled_spindles))
print(len(coupling_ends_train_filtered_downsampled_spindles))

print(len(coupling_starts_test_filtered_downsampled_spindles))
print(len(coupling_ends_test_filtered_downsampled_spindles))

NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
Filtering raw data in 73 contiguous segments
Setting up band-pass filter from 0.16 - 1.2 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.16
- Lower transition bandwidth: 0.16 Hz (-6 dB cutoff frequency: 0.08 Hz)
- Upper passband edge: 1.25 Hz
- Upper transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 2.25 Hz)
- Filter length: 10313 samples (20.626 s)

NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
Filtering raw data in 73 contiguous segments
Setting up band-pass filter from 0.16 - 1.2 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194

In [37]:
# Apply bandpass filter between 12 and 16 Hz
train_filtered_downsampled_spindles = train_raw.copy().filter(l_freq=12, h_freq=16)
test_filtered_downsampled_spindles = test_raw.copy().filter(l_freq=12, h_freq=16)

# Downsample to 100 Hz
train_filtered_downsampled_spindles.resample(100)
test_filtered_downsampled_spindles.resample(100)

Filtering raw data in 73 contiguous segments
Setting up band-pass filter from 12 - 16 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 12.00
- Lower transition bandwidth: 3.00 Hz (-6 dB cutoff frequency: 10.50 Hz)
- Upper passband edge: 16.00 Hz
- Upper transition bandwidth: 4.00 Hz (-6 dB cutoff frequency: 18.00 Hz)
- Filter length: 551 samples (1.102 s)

Filtering raw data in 50 contiguous segments
Setting up band-pass filter from 12 - 16 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 12.00
- Lower transition bandwidth: 3.00 Hz (-6 dB cutoff frequency: 10.50 Hz)
- Upper passban

<Raw | test_raw.fif, 1 x 3432010 (34320.1 s), ~26.2 MiB, data loaded>

### Create the epochs

In [38]:
epochs_train_filtered_downsampled_spindles = create_fixed_length_epochs(train_filtered_downsampled_spindles)
epochs_test_filtered_downsampled_spindles = create_fixed_length_epochs(test_filtered_downsampled_spindles)
print(epochs_train_filtered_downsampled_spindles.get_data().shape)
print(epochs_test_filtered_downsampled_spindles.get_data().shape)

Not setting metadata
14550 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 14550 events and 300 original time points ...
0 bad epochs dropped
Not setting metadata
11440 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 11440 events and 300 original time points ...
0 bad epochs dropped
(14550, 1, 300)
(11440, 1, 300)


### Label the epochs

In [39]:
%%time

# Train set

epoch_labels_train_filtered_downsampled_spindles = label_coupling_epochs(epochs_train_filtered_downsampled_spindles, coupling_starts_train_filtered_downsampled_spindles, coupling_ends_train_filtered_downsampled_spindles)

print(f"Train data first 60 coupling events: {coupling_starts_train_filtered_downsampled_spindles[:60]}")
print(f"Train labels first 60 epochs: {epoch_labels_train_filtered_downsampled_spindles[:60]}")

# Test set

epoch_labels_test_filtered_downsampled_spindles = label_coupling_epochs(epochs_test_filtered_downsampled_spindles, coupling_starts_test_filtered_downsampled_spindles, coupling_ends_test_filtered_downsampled_spindles)

print(f"\nTest data first 60 coupling events: {coupling_starts_test_filtered_downsampled_spindles[:60]}")
print(f"Test labels first 60 epochs: {epoch_labels_test_filtered_downsampled_spindles[:60]}")

Train data first 60 coupling events: (np.float64(83.41), np.float64(108.08), np.float64(262.01), np.float64(274.78), np.float64(378.52), np.float64(482.73), np.float64(502.23), np.float64(567.15), np.float64(599.39), np.float64(600.94), np.float64(685.84), np.float64(697.85), np.float64(707.43), np.float64(708.93), np.float64(720.49), np.float64(732.11), np.float64(776.09), np.float64(780.06), np.float64(864.6), np.float64(871.32), np.float64(910.43), np.float64(917.07), np.float64(960.41), np.float64(1050.52), np.float64(1200.05), np.float64(1262.04), np.float64(1278.35), np.float64(1330.87), np.float64(1356.47), np.float64(1371.44), np.float64(1395.83), np.float64(1449.16), np.float64(1566.73), np.float64(1615.49), np.float64(1616.67), np.float64(1973.78), np.float64(2039.95), np.float64(2091.33), np.float64(2121.41), np.float64(2244.99), np.float64(2244.99), np.float64(2367.83), np.float64(2368.8), np.float64(2440.8), np.float64(2445.04), np.float64(2565.68), np.float64(2910.09), np

In [40]:
%%time

# Train set

epoch_labels_train_filtered_downsampled_spindles_moderate = label_coupling_epochs_moderate(epochs_train_filtered_downsampled_spindles, coupling_starts_train_filtered_downsampled_spindles, coupling_ends_train_filtered_downsampled_spindles)

print(f"Train data first 60 coupling events: {coupling_starts_train_filtered_downsampled_spindles[:60]}")
print(f"Train labels first 60 epochs: {epoch_labels_train_filtered_downsampled_spindles_moderate[:60]}")

# Test set

epoch_labels_test_filtered_downsampled_spindles_moderate = label_coupling_epochs_moderate(epochs_test_filtered_downsampled_spindles, coupling_starts_test_filtered_downsampled_spindles, coupling_ends_test_filtered_downsampled_spindles)

print(f"\nTest data first 60 coupling events: {coupling_starts_test_filtered_downsampled_spindles[:60]}")
print(f"Test labels first 60 epochs: {epoch_labels_test_filtered_downsampled_spindles_moderate[:60]}")

Train data first 60 coupling events: (np.float64(83.41), np.float64(108.08), np.float64(262.01), np.float64(274.78), np.float64(378.52), np.float64(482.73), np.float64(502.23), np.float64(567.15), np.float64(599.39), np.float64(600.94), np.float64(685.84), np.float64(697.85), np.float64(707.43), np.float64(708.93), np.float64(720.49), np.float64(732.11), np.float64(776.09), np.float64(780.06), np.float64(864.6), np.float64(871.32), np.float64(910.43), np.float64(917.07), np.float64(960.41), np.float64(1050.52), np.float64(1200.05), np.float64(1262.04), np.float64(1278.35), np.float64(1330.87), np.float64(1356.47), np.float64(1371.44), np.float64(1395.83), np.float64(1449.16), np.float64(1566.73), np.float64(1615.49), np.float64(1616.67), np.float64(1973.78), np.float64(2039.95), np.float64(2091.33), np.float64(2121.41), np.float64(2244.99), np.float64(2244.99), np.float64(2367.83), np.float64(2368.8), np.float64(2440.8), np.float64(2445.04), np.float64(2565.68), np.float64(2910.09), np

In [41]:
print("With 50% threshold:")
# Train set
train_labels = epoch_labels_train_filtered_downsampled_spindles_moderate
train_counts = np.bincount(train_labels)
train_total = len(train_labels)

print("\nTrain label distribution:")
for label, count in enumerate(train_counts):
    proportion = count / train_total
    print(f"Label {label}: count = {count}, proportion = {proportion:.2%}")

# Test set
test_labels = epoch_labels_test_filtered_downsampled_spindles_moderate
test_counts = np.bincount(test_labels)
test_total = len(test_labels)

print("\nTest label distribution:")
for label, count in enumerate(test_counts):
    proportion = count / test_total
    print(f"Label {label}: count = {count}, proportion = {proportion:.2%}")

With 50% threshold:

Train label distribution:
Label 0: count = 13463, proportion = 92.53%
Label 1: count = 1087, proportion = 7.47%

Test label distribution:
Label 0: count = 10605, proportion = 92.70%
Label 1: count = 835, proportion = 7.30%


### Prepare EEG data for CNN input

#### X and y train and test sets

In [42]:
# Reshape arrays

epochs_train_np_filtered_downsampled_spindles = np.array(epochs_train_filtered_downsampled_spindles).reshape(len(epochs_train_filtered_downsampled_spindles), -1, 1)
# number of epochs N, sampling frequency (time dimension automatically inferred), channel dimension
epochs_test_np_filtered_downsampled_spindles = np.array(epochs_test_filtered_downsampled_spindles).reshape(len(epochs_test_filtered_downsampled_spindles), -1, 1)

In [43]:
# Define X and y sets
                                                                 
X_train_filtered_downsampled_spindles = epochs_train_np_filtered_downsampled_spindles
y_train_filtered_downsampled_spindles = epoch_labels_train_filtered_downsampled_spindles_moderate

X_test_filtered_downsampled_spindles = epochs_test_np_filtered_downsampled_spindles
y_test_filtered_downsampled_spindles = epoch_labels_test_filtered_downsampled_spindles_moderate

# Print shapes

print(f"X_train shape: {X_train_filtered_downsampled_spindles.shape}")
print(f"y_train shape: {y_train_filtered_downsampled_spindles.shape}")

print(f"\nX_test shape: {X_test_filtered_downsampled_spindles.shape}")
print(f"y_test shape: {y_test_filtered_downsampled_spindles.shape}")

X_train shape: (14550, 300, 1)
y_train shape: (14550,)

X_test shape: (11440, 300, 1)
y_test shape: (11440,)


## With STFT

In [44]:
coupling_train_times_stft_downsampled = detect_slow_oscillations_spindles_coupling_so_times(train_raw)
coupling_test_times_stft_downsampled = detect_slow_oscillations_spindles_coupling_so_times(test_raw)

coupling_starts_train_stft_downsampled, coupling_ends_train_stft_downsampled = zip(*coupling_train_times_stft_downsampled) if coupling_train_times_stft_downsampled else([],[])
coupling_starts_test_stft_downsampled, coupling_ends_test_stft_downsampled = zip(*coupling_test_times_stft_downsampled) if coupling_test_times_stft_downsampled else([],[])

print(len(coupling_starts_train_stft_downsampled))
print(len(coupling_ends_train_stft_downsampled))

print(len(coupling_starts_test_stft_downsampled))
print(len(coupling_ends_test_stft_downsampled))

NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
Filtering raw data in 73 contiguous segments
Setting up band-pass filter from 0.16 - 1.2 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.16
- Lower transition bandwidth: 0.16 Hz (-6 dB cutoff frequency: 0.08 Hz)
- Upper passband edge: 1.25 Hz
- Upper transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 2.25 Hz)
- Filter length: 10313 samples (20.626 s)

NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
Filtering raw data in 73 contiguous segments
Setting up band-pass filter from 0.16 - 1.2 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194

### Epoch the data

In [45]:
epochs_train_stft_downsampled = create_fixed_length_epochs(train_raw_downsampled)
epochs_test_stft_downsampled = create_fixed_length_epochs(test_raw_downsampled)

print(epochs_train_stft_downsampled.get_data().shape)
print(epochs_test_stft_downsampled.get_data().shape)

Not setting metadata
14550 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 14550 events and 300 original time points ...
0 bad epochs dropped
Not setting metadata
11440 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 11440 events and 300 original time points ...
0 bad epochs dropped
(14550, 1, 300)
(11440, 1, 300)


In [46]:
%%time

# Train set

epoch_labels_train_stft_downsampled = label_coupling_epochs(epochs_train_stft_downsampled, coupling_starts_train_stft_downsampled, coupling_ends_train_stft_downsampled)

print(f"Train data first 60 coupling events: {coupling_starts_train_stft_downsampled[:60]}")
print(f"Train labels first 60 epochs: {epoch_labels_train_stft_downsampled[:60]}")

# Test set

epoch_labels_test_stft_downsampled = label_coupling_epochs(epochs_test_stft_downsampled, coupling_starts_test_stft_downsampled, coupling_ends_test_stft_downsampled)

print(f"\nTest data first 60 coupling events: {coupling_starts_test_stft_downsampled[:60]}")
print(f"Test labels first 60 epochs: {epoch_labels_test_stft_downsampled[:60]}")

Train data first 60 coupling events: (np.float64(83.41), np.float64(108.08), np.float64(262.01), np.float64(274.78), np.float64(378.52), np.float64(482.73), np.float64(502.23), np.float64(567.15), np.float64(599.39), np.float64(600.94), np.float64(685.84), np.float64(697.85), np.float64(707.43), np.float64(708.93), np.float64(720.49), np.float64(732.11), np.float64(776.09), np.float64(780.06), np.float64(864.6), np.float64(871.32), np.float64(910.43), np.float64(917.07), np.float64(960.41), np.float64(1050.52), np.float64(1200.05), np.float64(1262.04), np.float64(1278.35), np.float64(1330.87), np.float64(1356.47), np.float64(1371.44), np.float64(1395.83), np.float64(1449.16), np.float64(1566.73), np.float64(1615.49), np.float64(1616.67), np.float64(1973.78), np.float64(2039.95), np.float64(2091.33), np.float64(2121.41), np.float64(2244.99), np.float64(2244.99), np.float64(2367.83), np.float64(2368.8), np.float64(2440.8), np.float64(2445.04), np.float64(2565.68), np.float64(2910.09), np

In [47]:
%%time

# Train set

epoch_labels_train_stft_downsampled_moderate = label_coupling_epochs_moderate(epochs_train_filtered_downsampled_spindles, coupling_starts_train_filtered_downsampled_spindles, coupling_ends_train_filtered_downsampled_spindles)

print(f"Train data first 60 coupling events: {coupling_starts_train_filtered_downsampled_spindles[:60]}")
print(f"Train labels first 60 epochs: {epoch_labels_train_filtered_downsampled_spindles_moderate[:60]}")

# Test set

epoch_labels_test_stft_downsampled_moderate = label_coupling_epochs_moderate(epochs_test_filtered_downsampled_spindles, coupling_starts_test_filtered_downsampled_spindles, coupling_ends_test_filtered_downsampled_spindles)

print(f"\nTest data first 60 coupling events: {coupling_starts_test_filtered_downsampled_spindles[:60]}")
print(f"Test labels first 60 epochs: {epoch_labels_test_filtered_downsampled_spindles_moderate[:60]}")

Train data first 60 coupling events: (np.float64(83.41), np.float64(108.08), np.float64(262.01), np.float64(274.78), np.float64(378.52), np.float64(482.73), np.float64(502.23), np.float64(567.15), np.float64(599.39), np.float64(600.94), np.float64(685.84), np.float64(697.85), np.float64(707.43), np.float64(708.93), np.float64(720.49), np.float64(732.11), np.float64(776.09), np.float64(780.06), np.float64(864.6), np.float64(871.32), np.float64(910.43), np.float64(917.07), np.float64(960.41), np.float64(1050.52), np.float64(1200.05), np.float64(1262.04), np.float64(1278.35), np.float64(1330.87), np.float64(1356.47), np.float64(1371.44), np.float64(1395.83), np.float64(1449.16), np.float64(1566.73), np.float64(1615.49), np.float64(1616.67), np.float64(1973.78), np.float64(2039.95), np.float64(2091.33), np.float64(2121.41), np.float64(2244.99), np.float64(2244.99), np.float64(2367.83), np.float64(2368.8), np.float64(2440.8), np.float64(2445.04), np.float64(2565.68), np.float64(2910.09), np

### Apply STFT to each epoch

#### get rid of channel dimension

In [48]:
epochs_train_stft_downsampled = np.squeeze(epochs_train_stft_downsampled)
epochs_test_stft_downsampled = np.squeeze(epochs_test_stft_downsampled)

print(epochs_train_stft_downsampled.shape)
print(epochs_test_stft_downsampled.shape)

(14550, 300)
(11440, 300)


In [49]:
print(train_raw_downsampled.info['sfreq'])

100.0


In [50]:
fs = train_raw_downsampled.info['sfreq']  
# smaller nperseg means higher time resolution
# noverlap must be less than nperseg

# I want to focus more on the frequency resolution
# at the expense of the time resolution
# for spindles, want a 1-2 Hz resolution (frequency bins every 1-2 Hz)
# let's start with 2 Hz first 
nperseg = 50
noverlap = nperseg // 2
# common practice is to set noverlap to 50% of nperseg


epochs_train_stft_transformed_downsampled = []

for epoch in epochs_train_stft_downsampled:
    f, t, Zxx = stft(epoch, fs=fs, nperseg=nperseg, noverlap=noverlap)
    spectrogram = np.abs(Zxx)  
    epochs_train_stft_transformed_downsampled.append(spectrogram)

epochs_test_stft_transformed_downsampled = []

for epoch in epochs_test_stft_downsampled:
    f, t, Zxx = stft(epoch, fs=fs, nperseg=nperseg, noverlap=noverlap)
    spectrogram = np.abs(Zxx)  
    epochs_test_stft_transformed_downsampled.append(spectrogram)
    
# convert into numpy arrays
epochs_train_stft_transformed_downsampled = np.array(epochs_train_stft_transformed_downsampled)
epochs_test_stft_transformed_downsampled = np.array(epochs_test_stft_transformed_downsampled)

print("Train STFT shape:", epochs_train_stft_transformed_downsampled.shape)
print("Test STFT shape:", epochs_test_stft_transformed_downsampled.shape)

# shape is number of epochs, frequency_bins, time_bins

Train STFT shape: (14550, 26, 13)
Test STFT shape: (11440, 26, 13)


In [51]:
# Define X and y sets and reshape

X_train_stft_downsampled = epochs_train_stft_transformed_downsampled[..., np.newaxis]  # Shape: (14550, 65, 25, 1)
y_train_stft_downsampled = epoch_labels_train_stft_downsampled_moderate

X_test_stft_downsampled = epochs_test_stft_transformed_downsampled[..., np.newaxis]    # Shape: (11440, 65, 25, 1)
y_test_stft_downsampled = epoch_labels_test_stft_downsampled_moderate
                                                                 

# Print shapes

print(f"X_train shape: {X_train_stft_downsampled.shape}")
print(f"y_train shape: {y_train_stft_downsampled.shape}")

print(f"\nX_test shape: {X_test_stft_downsampled.shape}")
print(f"y_test shape: {y_test_stft_downsampled.shape}")

X_train shape: (14550, 26, 13, 1)
y_train shape: (14550,)

X_test shape: (11440, 26, 13, 1)
y_test shape: (11440,)


### Normalization of data

In [52]:
# this scales the spectrogram in range 0,1
# this is min-max normalisation

X_train_stft_norm_downsampled = np.array([
    (epoch - np.min(epoch)) / (np.max(epoch) - np.min(epoch) + 1e-8)
    for epoch in X_train_stft_downsampled
])

X_test_stft_norm_downsampled = np.array([
    (epoch - np.min(epoch)) / (np.max(epoch) - np.min(epoch) + 1e-8)
    for epoch in X_test_stft_downsampled
])

In [53]:
# should have values between 0 and 1 

print("Before normalisation:")
print("Max train value:", np.max(X_train_stft_downsampled))
print("Min train value:", np.min(X_train_stft_downsampled))

print("Max test value:", np.max(X_test_stft_downsampled))
print("Min test value:", np.min(X_test_stft_downsampled))

print("\nAfter normalisation:")
print("Max train value:", np.max(X_train_stft_norm_downsampled))
print("Min train value:", np.min(X_train_stft_norm_downsampled))

print("Max test value:", np.max(X_test_stft_norm_downsampled))
print("Min test value:", np.min(X_test_stft_norm_downsampled))

Before normalisation:
Max train value: 866.3673727627713
Min train value: 2.4061107274064854e-06
Max test value: 1011.6755263560262
Min test value: 5.496909294180341e-06

After normalisation:
Max train value: 0.9999999999884571
Min train value: 0.0
Max test value: 0.9999999999901138
Min test value: 0.0


## Two input model

### Reducing STFT dimension

#### Only keeping frequency axis

In [54]:
X_train_stft_freq = np.mean(X_train_stft_norm_downsampled, axis=1) 
X_test_stft_freq = np.mean(X_test_stft_norm_downsampled, axis=1)

print(f"Shape of X_train_stft_freq:{X_train_stft_freq.shape}")
print(f"Shape of X_test_stft_freq:{X_test_stft_freq.shape}")

Shape of X_train_stft_freq:(14550, 13, 1)
Shape of X_test_stft_freq:(11440, 13, 1)


#### Two-input model with frequency for the STFT

In [55]:
X_train_dict_freq = {
    'raw_input': X_train_raw_downsampled,
    'stft_input': X_train_stft_freq
}

X_test_dict_freq = {
    'raw_input': X_test_raw_downsampled,
    'stft_input': X_test_stft_freq
}

In [56]:
cnn_model_multi_input_freq = build_multi_input_cnn_model_freq()
cnn_model_multi_input_freq.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ raw_input (InputLayer)        │ (None, 300, 1)            │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ stft_input (InputLayer)       │ (None, 13, 1)             │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ zero_padding1d_3              │ (None, 304, 1)            │               0 │ raw_input[0][0]            │
│ (ZeroPadding1D)               │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ zero_padding1d_4              │ (None, 310, 1)            │               0 │ raw_input[0][0]            │
│ (ZeroPadding1D)               │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ zero_padding1d_5              │ (None, 320, 1)            │               0 │ raw_input[0][0]            │
│ (ZeroPadding1D)               │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ zero_padding1d_6              │ (None, 17, 1)             │               0 │ stft_input[0][0]           │
│ (ZeroPadding1D)               │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ zero_padding1d_7              │ (None, 23, 1)             │               0 │ stft_input[0][0]           │
│ (ZeroPadding1D)               │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ zero_padding1d_8              │ (None, 33, 1)             │               0 │ stft_input[0][0]           │
│ (ZeroPadding1D)               │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv1d_3 (Conv1D)             │ (None, 300, 10)           │              60 │ zero_padding1d_3[0][0]     │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv1d_4 (Conv1D)             │ (None, 300, 10)           │             120 │ zero_padding1d_4[0][0]     │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv1d_5 (Conv1D)             │ (None, 300, 10)           │             220 │ zero_padding1d_5[0][0]     │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv1d_6 (Conv1D)             │ (None, 13, 10)            │              60 │ zero_padding1d_6[0][0]     │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv1d_7 (Conv1D)             │ (None, 13, 10)            │             120 │ zero_padding1d_7[0][0]     │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv1d_8 (Conv1D)             │ (None, 13, 10)            │             220 │ zero_padding1d_8[0][0]     │
├───────────────────────────────┼───────────────────────────┼───────────────

 Total params: 46,225 (180.57 KB)

 Trainable params: 46,105 (180.10 KB)

 Non-trainable params: 120 (480.00 B)

##### for early stopping

In [57]:
early_stop = EarlyStopping(
    monitor='val_loss',      
    patience=5,               
    restore_best_weights=True 
)
# stop after 5 epochs with no improvement

##### to balance class weights

In [58]:
# Assuming y_train_raw_downsampled is a 1D array of class labels
class_weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train_raw_downsampled),
    y=y_train_raw_downsampled
)
class_weight_dict = dict(enumerate(class_weights))

#### Training

In [59]:
%%time

training_info_multiple_inputs_freq = cnn_model_multi_input_freq.fit(X_train_dict_freq, y_train_raw_downsampled, validation_split=0.2, epochs=20, batch_size=128, class_weight=class_weight_dict)

Epoch 1/20
91/91 ━━━━━━━━━━━━━━━━━━━━ 18s 122ms/step - accuracy: 0.6047 - loss: 0.6195 - val_accuracy: 0.6691 - val_loss: 0.6277
Epoch 2/20
91/91 ━━━━━━━━━━━━━━━━━━━━ 11s 120ms/step - accuracy: 0.7364 - loss: 0.5051 - val_accuracy: 0.6206 - val_loss: 0.6231
Epoch 3/20
91/91 ━━━━━━━━━━━━━━━━━━━━ 11s 117ms/step - accuracy: 0.7310 - loss: 0.4769 - val_accuracy: 0.7364 - val_loss: 0.4713
Epoch 4/20
91/91 ━━━━━━━━━━━━━━━━━━━━ 11s 118ms/step - accuracy: 0.7387 - loss: 0.4309 - val_accuracy: 0.7498 - val_loss: 0.4471
Epoch 5/20
91/91 ━━━━━━━━━━━━━━━━━━━━ 10s 112ms/step - accuracy: 0.7382 - loss: 0.4475 - val_accuracy: 0.7808 - val_loss: 0.4316
Epoch 6/20
91/91 ━━━━━━━━━━━━━━━━━━━━ 10s 114ms/step - accuracy: 0.7385 - loss: 0.4224 - val_accuracy: 0.7838 - val_loss: 0.4357
Epoch 7/20
91/91 ━━━━━━━━━━━━━━━━━━━━ 11s 117ms/step - accuracy: 0.7628 - loss: 0.3877 - val_accuracy: 0.8216 - val_loss: 0.3711
Epoch 8/20
91/91 ━━━━━━━━━━━━━━━━━━━━ 11s 120ms/step - accuracy: 0.7558 - loss: 0.3947 - val_accu

In [60]:
def plot_training_history(training_info):
    fig, axs = plt.subplots(1, 2, figsize=(16, 5))
    axs[0].plot(training_info.history['loss'], label="training set")
    axs[0].plot(training_info.history['val_loss'], label="validation set")
    axs[0].set_xlabel("Epoch")
    axs[0].set_ylabel("Loss")
    axs[0].grid(True)
    axs[0].legend()
    try:
        axs[1].plot(training_info.history['accuracy'], label="training set")
        axs[1].plot(training_info.history['val_accuracy'], label="validation set")
        axs[1].set_xlabel("Epoch")
        axs[1].set_ylabel("Accuracy")
        axs[1].grid(True)
        axs[1].legend()
    except:
        pass
  
    fig.suptitle("Training History for two-input CNN model with frequency component of STFT for coupling detection", fontsize=16)
    plt.tight_layout(rect=[0, 0, 1, 0.95])
    plt.show()

plot_training_history(training_info_multiple_inputs_freq)

In [61]:
cnn_model_multi_input_freq.evaluate(X_test_dict_freq, y_test_raw_downsampled)

358/358 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - accuracy: 0.7623 - loss: 0.4520


[0.4609273076057434, 0.753496527671814]

In [62]:
y_pred = cnn_model_multi_input_freq.predict(X_test_dict_freq)
y_pred_labels = (y_pred > 0.5).astype(int)

print(confusion_matrix(y_test_raw_downsampled, y_pred_labels))
print(classification_report(y_test_raw_downsampled, y_pred_labels))

358/358 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step
[[7904 2701]
 [ 119  716]]
              precision    recall  f1-score   support

           0       0.99      0.75      0.85     10605
           1       0.21      0.86      0.34       835

    accuracy                           0.75     11440
   macro avg       0.60      0.80      0.59     11440
weighted avg       0.93      0.75      0.81     11440



In [63]:
# Get predictions
y_pred = cnn_model_multi_input_freq.predict(X_test_dict_freq)
y_pred_labels = (y_pred > 0.5).astype(int)

# Confusion matrix
cm = confusion_matrix(y_test_raw_downsampled, y_pred_labels)
cm_df = pd.DataFrame(cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

# Classification report as a dataframe
report = classification_report(y_test_raw_downsampled, y_pred_labels, output_dict=True)
report_df = pd.DataFrame(report).transpose()

# Confusion matrix plotted as heatmap
plt.figure(figsize=(6, 5))
sns.heatmap(cm_df, annot=True, fmt="d", cmap="Blues")
plt.title("Confusion Matrix for Coupling Detection \nusing a two-input CNN with frequency \ncomponent of STFT")
plt.ylabel("True Label")
plt.xlabel("Predicted Label")
plt.tight_layout()
plt.show()

# Classification report as a table
fig, ax = plt.subplots(figsize=(10, 4))
ax.axis('tight')
ax.axis('off')
table = ax.table(cellText=report_df.round(2).values,
                 colLabels=report_df.columns,
                 rowLabels=report_df.index,
                 cellLoc='center',
                 loc='center')
table.auto_set_font_size(False)
table.set_fontsize(10)
table.scale(1.2, 1.2)
plt.title("Classification Report for Coupling Detection \nusing a two-input CNN with frequency \ncomponent of STFT", fontsize=14)
plt.tight_layout()
plt.show()

358/358 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step


In [64]:
# Flatten in case y_pred has shape (n_samples, 1)
y_pred_proba = y_pred.ravel()

# Compute ROC curve and AUC
fpr, tpr, thresholds = roc_curve(y_test_raw_downsampled, y_pred_proba)
roc_auc = auc(fpr, tpr)

# Plotting
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f"ROC curve (AUC = {roc_auc:.2f})")
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Random chance')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Receiver Operating Characteristic for a two-input CNN model \nwith the STFT frequency component for coupling detection")
plt.legend(loc="lower right")
plt.grid(True)
plt.tight_layout()
plt.show()

## 4-input model

In [65]:
X_train_stft_freq = np.mean(X_train_stft_norm_downsampled, axis=1) 
X_test_stft_freq = np.mean(X_test_stft_norm_downsampled, axis=1)

print(f"Shape of X_train_stft_freq:{X_train_stft_freq.shape}")
print(f"Shape of X_test_stft_freq:{X_test_stft_freq.shape}")

Shape of X_train_stft_freq:(14550, 13, 1)
Shape of X_test_stft_freq:(11440, 13, 1)


In [66]:
X_train_dict_freq_four_input = {
    'raw_input': X_train_raw_downsampled,
    'filtered_so': X_train_filtered_downsampled_so,
    'filtered_spindles': X_train_filtered_downsampled_spindles,
    'stft_input': X_train_stft_freq
}

X_test_dict_freq_four_input = {
    'raw_input': X_test_raw_downsampled,
    'filtered_so': X_test_filtered_downsampled_so,
    'filtered_spindles': X_test_filtered_downsampled_spindles,
    'stft_input': X_test_stft_freq
}

In [67]:
cnn_model_four_input_freq = build_four_input_cnn_model_freq()
cnn_model_four_input_freq.summary()

C:\Users\roseb\anaconda3\envs\msc_research_project\Lib\site-packages\keras\src\layers\activations\leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ raw_input (InputLayer)        │ (None, 300, 1)            │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ filtered_so (InputLayer)      │ (None, 300, 1)            │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ filtered_spindles             │ (None, 300, 1)            │               0 │ -                          │
│ (InputLayer)                  │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ stft_input (InputLayer)       │ (None, 13, 1)             │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ zero_padding1d_9              │ (None, 304, 1)            │               0 │ raw_input[0][0]            │
│ (ZeroPadding1D)               │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ zero_padding1d_10             │ (None, 310, 1)            │               0 │ raw_input[0][0]            │
│ (ZeroPadding1D)               │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ zero_padding1d_11             │ (None, 320, 1)            │               0 │ raw_input[0][0]            │
│ (ZeroPadding1D)               │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ zero_padding1d_12             │ (None, 304, 1)            │               0 │ filtered_so[0][0]          │
│ (ZeroPadding1D)               │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ zero_padding1d_13             │ (None, 310, 1)            │               0 │ filtered_so[0][0]          │
│ (ZeroPadding1D)               │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ zero_padding1d_14             │ (None, 320, 1)            │               0 │ filtered_so[0][0]          │
│ (ZeroPadding1D)               │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ zero_padding1d_15             │ (None, 304, 1)            │               0 │ filtered_spindles[0][0]    │
│ (ZeroPadding1D)               │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ zero_padding1d_16             │ (None, 310, 1)            │               0 │ filtered_spindles[0][0]    │
│ (ZeroPadding1D)               │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ zero_padding1d_17             │ (None, 320, 1)            │               

 Total params: 92,321 (360.63 KB)

 Trainable params: 92,081 (359.69 KB)

 Non-trainable params: 240 (960.00 B)

In [68]:
# Assuming y_train_raw_downsampled is a 1D array of class labels
class_weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train_raw_downsampled),
    y=y_train_raw_downsampled
)
class_weight_dict = dict(enumerate(class_weights))

### Training

In [69]:
%%time

training_info_four_inputs_freq = cnn_model_four_input_freq.fit(X_train_dict_freq_four_input, y_train_raw_downsampled, validation_split=0.2, epochs=20, batch_size=128)

Epoch 1/20
91/91 ━━━━━━━━━━━━━━━━━━━━ 45s 366ms/step - accuracy: 0.8945 - loss: 0.3031 - val_accuracy: 0.9419 - val_loss: 0.2263
Epoch 2/20
91/91 ━━━━━━━━━━━━━━━━━━━━ 33s 366ms/step - accuracy: 0.9191 - loss: 0.2165 - val_accuracy: 0.9419 - val_loss: 0.1986
Epoch 3/20
91/91 ━━━━━━━━━━━━━━━━━━━━ 34s 371ms/step - accuracy: 0.9243 - loss: 0.2016 - val_accuracy: 0.9430 - val_loss: 0.1804
Epoch 4/20
91/91 ━━━━━━━━━━━━━━━━━━━━ 34s 373ms/step - accuracy: 0.9200 - loss: 0.1964 - val_accuracy: 0.9419 - val_loss: 0.1633
Epoch 5/20
91/91 ━━━━━━━━━━━━━━━━━━━━ 35s 385ms/step - accuracy: 0.9237 - loss: 0.1928 - val_accuracy: 0.9436 - val_loss: 0.1747
Epoch 6/20
91/91 ━━━━━━━━━━━━━━━━━━━━ 46s 509ms/step - accuracy: 0.9238 - loss: 0.1896 - val_accuracy: 0.9436 - val_loss: 0.1502
Epoch 7/20
91/91 ━━━━━━━━━━━━━━━━━━━━ 44s 481ms/step - accuracy: 0.9235 - loss: 0.1864 - val_accuracy: 0.9440 - val_loss: 0.1392
Epoch 8/20
91/91 ━━━━━━━━━━━━━━━━━━━━ 48s 522ms/step - accuracy: 0.9234 - loss: 0.1846 - val_accu

In [70]:
def plot_training_history(training_info):
    fig, axs = plt.subplots(1, 2, figsize=(16, 5))
    axs[0].plot(training_info.history['loss'], label="training set")
    axs[0].plot(training_info.history['val_loss'], label="validation set")
    axs[0].set_xlabel("Epoch")
    axs[0].set_ylabel("Loss")
    axs[0].grid(True)
    axs[0].legend()
    try:
        axs[1].plot(training_info.history['accuracy'], label="training set")
        axs[1].plot(training_info.history['val_accuracy'], label="validation set")
        axs[1].set_xlabel("Epoch")
        axs[1].set_ylabel("Accuracy")
        axs[1].grid(True)
        axs[1].legend()
    except:
        pass
  
    fig.suptitle("Training History for a four-input CNN model with frequency component of STFT for coupling detection", fontsize=16)
    plt.tight_layout(rect=[0, 0, 1, 0.95])
    plt.show()

plot_training_history(training_info_four_inputs_freq)

In [71]:
cnn_model_four_input_freq.evaluate(X_test_dict_freq_four_input, y_test_raw_downsampled)

358/358 ━━━━━━━━━━━━━━━━━━━━ 5s 15ms/step - accuracy: 0.9225 - loss: 0.1824


[0.1663653552532196, 0.9298951029777527]

In [72]:
y_pred = cnn_model_four_input_freq.predict(X_test_dict_freq_four_input)
y_pred_labels = (y_pred > 0.5).astype(int)

print(confusion_matrix(y_test_raw_downsampled, y_pred_labels))
print(classification_report(y_test_raw_downsampled, y_pred_labels))

358/358 ━━━━━━━━━━━━━━━━━━━━ 7s 17ms/step
[[10328   277]
 [  525   310]]
              precision    recall  f1-score   support

           0       0.95      0.97      0.96     10605
           1       0.53      0.37      0.44       835

    accuracy                           0.93     11440
   macro avg       0.74      0.67      0.70     11440
weighted avg       0.92      0.93      0.92     11440



In [76]:
# Get predictions
y_pred = cnn_model_four_input_freq.predict(X_test_dict_freq_four_input)
y_pred_labels = (y_pred > 0.5).astype(int)

# Confusion matrix
cm = confusion_matrix(y_test_raw_downsampled, y_pred_labels)
cm_df = pd.DataFrame(cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

# Classification report as a dataframe
report = classification_report(y_test_raw_downsampled, y_pred_labels, output_dict=True)
report_df = pd.DataFrame(report).transpose()

# Confusion matrix plotted as heatmap
plt.figure(figsize=(6, 5))
sns.heatmap(cm_df, annot=True, fmt="d", cmap="Blues")
plt.title("Confusion Matrix for Coupling Detection \nusing a two-input CNN with frequency \ncomponent of STFT")
plt.ylabel("True Label")
plt.xlabel("Predicted Label")
plt.tight_layout()
plt.show()

# Classification report as a table
fig, ax = plt.subplots(figsize=(10, 4))
ax.axis('tight')
ax.axis('off')
table = ax.table(cellText=report_df.round(2).values,
                 colLabels=report_df.columns,
                 rowLabels=report_df.index,
                 cellLoc='center',
                 loc='center')
table.auto_set_font_size(False)
table.set_fontsize(10)
table.scale(1.2, 1.2)
plt.title("Classification Report for Coupling Detection \nusing a four-input CNN with frequency \ncomponent of STFT", fontsize=14)
plt.tight_layout()
plt.show()

358/358 ━━━━━━━━━━━━━━━━━━━━ 8s 22ms/step


In [74]:
# Flatten in case y_pred has shape (n_samples, 1)
y_pred_proba = y_pred.ravel()

# Compute ROC curve and AUC
fpr, tpr, thresholds = roc_curve(y_test_raw_downsampled, y_pred_proba)
roc_auc = auc(fpr, tpr)

# Plotting
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f"ROC curve (AUC = {roc_auc:.2f})")
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Random chance')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Receiver Operating Characteristic for a four-input CNN model \nwith the STFT frequency component for coupling detection")
plt.legend(loc="lower right")
plt.grid(True)
plt.tight_layout()
plt.show()